In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

!pip install transformers==4.26.0 datasets botocore==1.31.17
!pip install huggingface 
!pip install jsonlines config #lamini 
!pip install 'transformers[torch]'==4.26.0
!pip install git+https://github.com/huggingface/accelerate

In [ ]:
from llama import BasicModelRunner

model = BasicModelRunner("EleutherAI/pythia-410m") 
model.load_data_from_jsonlines("/kaggle/input/data-set/seed_lamini_docs.jsonl", input_key="question", output_key="answer")
# model.train(is_public=True) 

In [ ]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

#from utilities import *
from transformers import AutoTokenizer
#from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
#from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

In [ ]:
dataset_name = "lamini_docs.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = True

In [ ]:
dataset_path = "lamini/lamini_docs"
use_hf = True

Set up the model, training config, and tokenize

In [ ]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")
    


device

In [ ]:
model_name = "facebook/opt-350m" #"EleutherAI/pythia-70m-deduped"#"EleutherAI/pythia-70m-deduped"
from transformers import GPTNeoXForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained(   #GPTNeoXForCausalLM
  model_name,
  #revision="step3000",
  cache_dir="./pythia-70m-deduped/step3000",
).to(device)

tokenizer = AutoTokenizer.from_pretrained(
  model_name,
  #revision="step3000",
  cache_dir="./pythia-70m-deduped/step3000",
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
      if len(examples["question"][0].strip()) == 0:   #examples["question"][0] =='' or 
        examples["question"][0] ='#'
      if len(examples["answer"][0].strip()) == 0:
        examples["answer"][0] ='#'
      text = examples["question"][0] + examples["answer"][0]
    elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
    else:
      text = examples["text"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length,
        padding=True
    )
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'][:]
    return tokenized_inputs

In [ ]:
filename= "/kaggle/input/data-set/seed_lamini_docs.jsonl"
finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")
# finetuning_dataset_loaded = datasets.load_dataset("kotzeje/tokenized_lamini_docs")
tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

In [ ]:
print(tokenized_dataset[0])

In [ ]:
df= pd.DataFrame(tokenized_dataset)

df_filtered = df[df['question'] == '#'].index
# questdf_filteredon=df_filtered.fillna("NaN")
# df[modifiedDf['column_name'] == ''].index

df.drop(df_filtered , inplace=True)
print(df.columns)
#df.drop(columns=['__index_level_0__'],inplace=True)
from datasets import Dataset
tokenized_dataset= Dataset.from_pandas(df)
tokenized_dataset = tokenized_dataset.remove_columns('__index_level_0__')
tokenized_dataset

In [ ]:
#This was the bogus step 
#tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.12, shuffle=True, seed=11)

#print(split_dataset)
train_dataset, test_dataset =split_dataset['train'] , split_dataset['test'] 
print(train_dataset)
print(test_dataset)



In [ ]:
print( train_dataset['input_ids'][960][0], train_dataset['labels'][960][0] )
#print( len(train_dataset['input_ids'][22][0]), len(train_dataset['labels'][22][0]) )

**Load the base model**

In [ ]:
#base_model = AutoModelForCausalLM.from_pretrained(model_name)

**Define function to carry out inference**

**Setup training**

In [ ]:
!pip install evaluate nltk rouge_score

In [ ]:
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
# Setup evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result
def compute_rouge_score(generated, reference):
    """
    Compute ROUGE scores on a batch of articles.

    This is a convenience function wrapping Hugging Face `rouge_score`,
    which expects sentences to be separated by newlines.

    :param generated: Summaries (list of strings) produced by the model
    :param reference: Ground-truth summaries (list of strings) for comparison
    """
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    return metric.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
    )

In [ ]:
max_steps ='3-epochs'
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.5e-5,

  # Number of training epochs
  num_train_epochs=3,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  #max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  #optim="adamw_torch",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=3,
  metric_for_best_model= "eval_loss",
  greater_is_better=False,

#
  weight_decay=0.01,
  
)

In [ ]:
use_hf = False
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf, #use_hf = False
        "path": filename    #filename= "/kaggle/input/data-set/seed_lamini_docs.jsonl"
    },
    "verbose": True
}

model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
from transformers import Trainer
trainer = Trainer(
    model=base_model,
#     model_flops=model_flops,
#     total_steps=max_steps,
    args= training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
#     compute_metrics=compute_rouge_score, 
)

In [ ]:
import os
trainer.train()

In [ ]:
output_dir = '/kaggle/input/'
save_dir = '/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

In [ ]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

In [ ]:
finetuned_slightly_model.to(device) 

In [ ]:
def inference(text, model, tokenizer, max_input_tokens=1024, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
#   generated_tokens_with_prompt = model.generate(
#     input_ids=input_ids.to(device),
#     #max_length=max_output_tokens,
#     max_new_tokens=max_output_tokens,
#     pad_token_id=tokenizer.eos_token_id,
#     eos_token_id=model.config.eos_token_id,
#     output_scores=True,
#     num_beams=5,
#     #num_return_sequences=3,
    
#     # num_beam_groups=5,
#     # diversity_penalty=1.0,
    
#     no_repeat_ngram_size=3,
#     early_stopping=True,
#     #temperature=0.6,
#     do_sample=True,

#   )

  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens,
    #max_new_tokens=max_output_tokens,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=model.config.eos_token_id,
    output_scores=True,
    penalty_alpha=0.4, top_k=4,
    num_beams=5,
    #num_return_sequences=2,
    
    # num_beam_groups=5,
    # diversity_penalty=1.0,
    
    no_repeat_ngram_size=3,
    early_stopping=True,
    #temperature=0.6,
    do_sample=True,
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [ ]:
from pprint import pprint
test_question = test_dataset[55]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
generated = inference(test_question, finetuned_slightly_model, tokenizer)
pprint(generated)

In [ ]:
test_answer = test_dataset[25]['answer']
print("Target answer output (test):", test_answer)

In [ ]:
# TODO
rouge_scores = compute_rouge_score(generated, test_answer)

In [ ]:
!pip install autotrain-advanced